In [ ]:
!pip install numpy==1.23.0 --force

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 40.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.0 which is incompatible.


In [ ]:
import pandas as pd
class PSAR:

  def __init__(self, init_af=0.02, max_af=0.2, af_step=0.02):
    self.max_af = max_af
    self.init_af = init_af
    self.af = init_af
    self.af_step = af_step
    self.extreme_point = None
    self.high_price_trend = []
    self.low_price_trend = []
    self.high_price_window = deque(maxlen=2)
    self.low_price_window = deque(maxlen=2)

    self.psar_list = []
    self.af_list = []
    self.ep_list = []
    self.high_list = []
    self.low_list = []
    self.trend_list = []
    self._num_days = 0

  def calcPSAR(self, high, low):
    if self._num_days >= 3:
      psar = self._calcPSAR()
    else:
      psar = self._initPSARVals(high, low)

    psar = self._updateCurrentVals(psar, high, low)
    self._num_days += 1

    return psar

  def _initPSARVals(self, high, low):
    if len(self.low_price_window) <= 1:
      self.trend = None
      self.extreme_point = high
      return None

    if self.high_price_window[0] < self.high_price_window[1]:
      self.trend = 1
      psar = min(self.low_price_window)
      self.extreme_point = max(self.high_price_window)
    else:
      self.trend = 0
      psar = max(self.high_price_window)
      self.extreme_point = min(self.low_price_window)

    return psar

  def _calcPSAR(self):
    prev_psar = self.psar_list[-1]
    if self.trend == 1: # Up
      psar = prev_psar + self.af * (self.extreme_point - prev_psar)
      psar = min(psar, min(self.low_price_window))
    else:
      psar = prev_psar - self.af * (prev_psar - self.extreme_point)
      psar = max(psar, max(self.high_price_window))

    return psar

  def _updateCurrentVals(self, psar, high, low):
    if self.trend == 1:
      self.high_price_trend.append(high)
    elif self.trend == 0:
      self.low_price_trend.append(low)

    psar = self._trendReversal(psar, high, low)

    self.psar_list.append(psar)
    self.af_list.append(self.af)
    self.ep_list.append(self.extreme_point)
    self.high_list.append(high)
    self.low_list.append(low)
    self.high_price_window.append(high)
    self.low_price_window.append(low)
    self.trend_list.append(self.trend)

    return psar

  def _trendReversal(self, psar, high, low):
    # Checks for reversals
    reversal = False
    if self.trend == 1 and psar > low:
      self.trend = 0
      psar = max(self.high_price_trend)
      self.extreme_point = low
      reversal = True
    elif self.trend == 0 and psar < high:
      self.trend = 1
      psar = min(self.low_price_trend)
      self.extreme_point = high
      reversal = True

    if reversal:
      self.af = self.init_af
      self.high_price_trend.clear()
      self.low_price_trend.clear()
    else:
        if high > self.extreme_point and self.trend == 1:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = high
        elif low < self.extreme_point and self.trend == 0:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = low

    return psar

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
!pip install yfinance

(Reading database ... 121692 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

  Using cached TA-Lib-0.4.28.tar.gz (357 kB)
  Installing build dependenci

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
# import talib as ta
from scipy.signal import argrelextrema
from collections import deque

def rsiuse(dfa1):
    global data,hh,lh,ll,hl,hh_idx,lh_idx,ll_idx,hl_idx,price,dates
    data=dfa1
    price = data['Close'].values
    dates = data.index
    # Get higher highs, lower lows, etc.
    order = 5
    hh = getHigherHighs(price, order)
    lh = getLowerHighs(price, order)
    ll = getLowerLows(price, order)
    hl = getHigherLows(price, order)
    # Get confirmation indices
    hh_idx = np.array([i[1] + order for i in hh])
    lh_idx = np.array([i[1] + order for i in lh])
    ll_idx = np.array([i[1] + order for i in ll])
    hl_idx = np.array([i[1] + order for i in hl])
    return data
def getHigherLows(data: np.array, order=5, K=2):
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are higher than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] < lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerHighs(data: np.array, order=5, K=2):
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are lower than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] > highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getHigherHighs(data: np.array, order=5, K=2):
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are higher than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] < highs[i-1]:
      ex_deque.clear()
    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerLows(data: np.array, order=5, K=2):
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are lower than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] > lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def calcRSI(data, P=14):
  data['diff_close'] = data['Close'] - data['Close'].shift(1)
  data['gain'] = np.where(data['diff_close']>0, data['diff_close'], 0)
  data['loss'] = np.where(data['diff_close']<0, np.abs(data['diff_close']), 0)
  data[['init_avg_gain', 'init_avg_loss']] = data[
    ['gain', 'loss']].rolling(P).mean()
  avg_gain = np.zeros(len(data))
  avg_loss = np.zeros(len(data))
  for i, _row in enumerate(data.iterrows()):
    row = _row[1]
    if i < P - 1:
      last_row = row.copy()
      continue
    elif i == P-1:
      avg_gain[i] += row['init_avg_gain']
      avg_loss[i] += row['init_avg_loss']
    else:
      avg_gain[i] += ((P - 1) * avg_gain[i-1] + row['gain']) / P
      avg_loss[i] += ((P - 1) * avg_loss[i-1] + row['loss']) / P

    last_row = row.copy()

  data['avg_gain'] = avg_gain
  data['avg_loss'] = avg_loss
  data['RS'] = data['avg_gain'] / data['avg_loss']
  data['RSI'] = 100 - 100 / (1 + data['RS'])
  return data
def getHHIndex(data: np.array, order=5, K=2):
  extrema = getHigherHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLHIndex(data: np.array, order=5, K=2):
  extrema = getLowerHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLLIndex(data: np.array, order=5, K=2):
  extrema = getLowerLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getHLIndex(data: np.array, order=5, K=2):
  extrema = getHigherLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getPeaks(data, key='Close', order=5, K=2):
  vals = data[key].values
  hh_idx = getHHIndex(vals, order, K)
  lh_idx = getLHIndex(vals, order, K)
  ll_idx = getLLIndex(vals, order, K)
  hl_idx = getHLIndex(vals, order, K)

  data[f'{key}_highs'] = np.nan
  data[f'{key}_highs'][hh_idx] = 1
  data[f'{key}_highs'][lh_idx] = -1
  data[f'{key}_highs'] = data[f'{key}_highs'].ffill().fillna(0)
  data[f'{key}_lows'] = np.nan
  data[f'{key}_lows'][ll_idx] = 1
  data[f'{key}_lows'][hl_idx] = -1
  data[f'{key}_lows'] = data[f'{key}_highs'].ffill().fillna(0)
  return data
def RSIDivergenceStrategy(data, P=14, order=5, K=2):
  data = getPeaks(data, key='Close', order=order, K=K)
  data = calcRSI(data, P=P)
  data = getPeaks(data, key='RSI', order=order, K=K)

  position = np.zeros(data.shape[0])
  for i, (t, row) in enumerate(data.iterrows()):
    if np.isnan(row['RSI']):
        continue
    # If no position is on
    if position[i-1] == 0:
      # Buy if indicator to higher low and price to lower low
      if row['Close_lows'] == -1 and row['RSI_lows'] == 1:
        if row['RSI'] < 50:
          position[i] = 1
          entry_rsi = row['RSI'].copy()

      # Short if price to higher high and indicator to lower high
      elif row['Close_highs'] == 1 and row['RSI_highs'] == -1:
        if row['RSI'] > 50:
          position[i] = -1
          entry_rsi = row['RSI'].copy()

    # If current position is long
    elif position[i-1] == 1:
        if row['RSI'] < 50 and row['RSI'] < entry_rsi:
            position[i] = 1

    # If current position is short
    elif position[i-1] == -1:
        if row['RSI'] < 50 and row['RSI'] > entry_rsi:
            position[i] = -1

  data['position'] = position
  return calcReturns(data)

def calcReturns(df):
  # Helper function to avoid repeating too much code
  df['returns'] = df['Close'] / df['Close'].shift(1)
  df['log_returns'] = np.log(df['returns'])
  df['strat_returns'] = df['position'].shift(1) * df['returns']
  df['strat_log_returns'] = df['position'].shift(1) * df['log_returns']
  df['cum_returns'] = np.exp(df['log_returns'].cumsum()) - 1
  df['strat_cum_returns'] = np.exp(df['strat_log_returns'].cumsum()) - 1
  df['peak'] = df['cum_returns'].cummax()
  df['strat_peak'] = df['strat_cum_returns'].cummax()
  return df
def rsidiver():
    position_df = RSIDivergenceStrategy(data)
    positions = position_df['position'].tolist()
    rsi_divergent_signal=positions
    return rsi_divergent_signal

In [ ]:
def findindi(Name,Real_name,Syear,Smonth,Sday,Eyear,Emonth,Eday):
    import datetime
    import pandas_datareader.data as web
    import matplotlib.pyplot as plt
    import numpy as np
    import talib as ta
    import pandas as pd
    import yfinance as yf
    from collections import deque
    import pandas_datareader as pdr
    from scipy import stats
    from scipy import signal as sg
    import plotly.express as px
    import plotly.graph_objects as go
    global obv,dfa1_all
	#preprocess data
    Sdate ="%s-%s-%s" % (Syear,Smonth,Sday)
    Edate ="%s-%s-%s" % (Eyear,Emonth,Eday)
    start=datetime.datetime(Syear,Smonth,Sday)
    end=datetime.datetime(Eyear,Emonth,Eday)
    Stockdata=yf.download(Name,start,end)
    dfa1=Stockdata
    Byear = Syear-1
    start=datetime.datetime(Byear,Smonth,Sday)
    end=datetime.datetime(Eyear,Emonth,Eday)
    Stockdata=yf.download(Name,start,end)
    dfa1_all=Stockdata

    def psar_changetrend_signal_func():
        global psar_changetrend_signal
        ticker = Name
        yfObj = yf.Ticker(ticker)
        yfO = yfObj.history(start=Sdate, end=Edate)[['High', 'Low']].round(2)
        yfO=yfO.loc[Sdate:Edate]
        yfO.reset_index(inplace=True)
        indic = PSAR()
        yfO['PSAR'] = yfO.apply(lambda x: indic.calcPSAR(x['High'], x['Low']), axis=1)
        yfO['Trend'] = indic.trend_list
        indic._calcPSAR()
        psar_changetrend_signal=[0]*len(yfO['Trend'])
        for idx,i in enumerate(yfO['Trend']):
            if (pd.isna(i) or idx==0):
                psar_changetrend_signal[idx]=0
            else:
                if (yfO['Trend'][idx-1]==0 and yfO['Trend'][idx]==1):
                    psar_changetrend_signal[idx]=1
                elif(yfO['Trend'][idx-1]==1 and yfO['Trend'][idx]==0):
                    psar_changetrend_signal[idx]=-1
                else:
                    psar_changetrend_signal[idx]=0
    psar_changetrend_signal_func()

    def psar_trend_signal_func():
        global psar_trend_signal
        ticker = Name
        yfObj = yf.Ticker(ticker)
        yfO = yfObj.history(start=Sdate, end=Edate)[['High', 'Low']].round(2)
        yfO=yfO.loc[Sdate:Edate]
        yfO.reset_index(inplace=True)
        indic = PSAR()
        yfO['PSAR'] = yfO.apply(lambda x: indic.calcPSAR(x['High'], x['Low']), axis=1)
        yfO['Trend'] = indic.trend_list
        indic._calcPSAR()
        psar_trend_signal=[0]*len(yfO['Trend'])
        for idx,i in enumerate(yfO['Trend']):
            if (yfO['Trend'][idx]==1):
                psar_trend_signal[idx]=1
            elif(yfO['Trend'][idx]==0):
                psar_trend_signal[idx]=-1
            else:
                psar_trend_signal[idx]=0
    psar_trend_signal_func()

    MA30=ta.MA(dfa1_all['Close'], timeperiod=30, matype=0)
    MA30=MA30.loc[Sdate:Edate]
    MA30=MA30.array
    dfa1['MA30']=MA30

    MA50=ta.MA(dfa1_all['Close'], timeperiod=50, matype=0)
    MA50=MA50.loc[Sdate:Edate]
    MA50=MA50.array
    dfa1['MA50']=MA50

    MA200=ta.MA(dfa1_all['Close'], timeperiod=200, matype=0)
    MA200=MA200.loc[Sdate:Edate]
    MA200=MA200.array
    dfa1['MA200']=MA200

    MA100=ta.MA(dfa1_all['Close'], timeperiod=100, matype=0)
    MA100=MA100.loc[Sdate:Edate]
    MA100=MA100.array
    dfa1['MA100']=MA100

    MA25=ta.MA(dfa1_all['Close'], timeperiod=25, matype=0)
    MA25=MA25.loc[Sdate:Edate]
    MA25=MA25.array
    dfa1['MA25']=MA25

    MA89=ta.MA(dfa1_all['Close'], timeperiod=89, matype=0)
    MA89=MA89.loc[Sdate:Edate]
    MA89=MA89.array
    dfa1['MA89']=MA89

    MA21=ta.MA(dfa1_all['Close'], timeperiod=21, matype=0)
    MA21=MA21.loc[Sdate:Edate]
    MA21=MA21.array
    dfa1['MA21']=MA21

    ADX=ta.ADX(dfa1_all['High'], dfa1_all['Low'], dfa1_all['Close'], timeperiod=14)
    ADX=ADX.loc[Sdate:Edate]
    ADX=ADX.array
    dfa1['ADX']=ADX
    plus_DMI = ta.PLUS_DM(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
    minus_DMI= ta.MINUS_DM(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
    plus_DMI=plus_DMI.loc[Sdate:Edate]
    minus_DMI=minus_DMI.loc[Sdate:Edate]
    plus_DMI=plus_DMI.array
    minus_DMI=minus_DMI.array
    dfa1['plus_DMI']=plus_DMI
    dfa1['minus_DMI']=minus_DMI
    prices, pdi, ndi, adx=dfa1['Close'], dfa1['plus_DMI'], dfa1['minus_DMI'], dfa1['ADX']

    def DMI_signal_func():
        global DMI_signal
        signal=0
        DMI_signal=[]
        for i in range(len(dfa1)):
            if plus_DMI[i-1] < minus_DMI[i-1] and plus_DMI[i] > minus_DMI[i]:
                DMI_signal.append(1)
            elif plus_DMI[i-1] > minus_DMI[i-1] and plus_DMI[i] < minus_DMI[i]:
                DMI_signal.append(-1)
            else:
                DMI_signal.append(0)

    def ADX_DMI_signal_func():
        global ADX_DMI_signal
        signal=0
        ADX_DMI_signal=[]
        for i in range(len(dfa1)):
            if ADX[i]>75 and ADX[i]<ADX[i-1] and plus_DMI[i] > minus_DMI[i]:
                ADX_DMI_signal.append(-1)
            elif plus_DMI[i-1] < minus_DMI[i-1] and plus_DMI[i] > minus_DMI[i]:
                ADX_DMI_signal.append(1)
            else:
                ADX_DMI_signal.append(0)

    MA5=ta.MA(dfa1_all['Close'], timeperiod=5, matype=0)
    MA5=MA5.loc[Sdate:Edate]
    MA5=MA5.array
    dfa1['MA5']=MA5

    MA10=ta.MA(dfa1_all['Close'], timeperiod=10, matype=0)
    MA10=MA10.loc[Sdate:Edate]
    MA10=MA10.array
    dfa1['MA10']=MA10

    MA12=ta.MA(dfa1_all['Close'], timeperiod=12, matype=0)
    MA12=MA12.loc[Sdate:Edate]
    MA12=MA12.array
    dfa1['MA12']=MA12

    MA21=ta.MA(dfa1_all['Close'], timeperiod=21, matype=0)
    MA21=MA21.loc[Sdate:Edate]
    MA21=MA21.array
    dfa1['MA12']=MA12

    MA7=ta.MA(dfa1_all['Close'], timeperiod=12, matype=0)
    MA7=MA7.loc[Sdate:Edate]
    MA7=MA7.array
    dfa1['MA7']=MA7

    MA16=ta.MA(dfa1_all['Close'], timeperiod=21, matype=0)
    MA16=MA16.loc[Sdate:Edate]
    MA16=MA16.array
    dfa1['MA16']=MA16

    obv = ta.OBV(dfa1_all["Close"], dfa1_all["Volume"])
    obv_mean=obv.rolling(5).mean()
    obv_mean=obv_mean.loc[Sdate:Edate]
    data=dfa1['Open']

    def ma12_21_signal_func():
        global ma12_21_signal
        signal=0
        ma12_21_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA12'][i] > dfa1['MA21'][i]:
                if signal != 1:
                    signal = 1
                    ma12_21_signal.append(signal)
                else:
                    ma12_21_signal.append(0)
            elif dfa1['MA12'][i] < dfa1['MA21'][i]:
                if signal != -1:
                    signal = -1
                    ma12_21_signal.append(signal)
                else:
                    ma12_21_signal.append(0)
            else:
                ma12_21_signal.append(0)

    def ma7_16_signal_func():
        global ma7_16_signal
        signal=0
        ma7_16_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA7'][i] > dfa1['MA16'][i]:
                if signal != 1:
                    signal = 1
                    ma7_16_signal.append(signal)
                else:
                    ma7_16_signal.append(0)
            elif dfa1['MA7'][i] < dfa1['MA16'][i]:
                if signal != -1:
                    signal = -1
                    ma7_16_signal.append(signal)
                else:
                    ma7_16_signal.append(0)
            else:
                ma7_16_signal.append(0)


    def ma25_89_signal_func():
        global ma25_89_signal
        signal=0
        ma25_89_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA25'][i] > dfa1['MA89'][i]:
                if signal != 1:
                    signal = 1
                    ma25_89_signal.append(signal)
                else:
                    ma25_89_signal.append(0)
            elif dfa1['MA25'][i] < dfa1['MA89'][i]:
                if signal != -1:
                    signal = -1
                    ma25_89_signal.append(signal)
                else:
                    ma25_89_signal.append(0)
            else:
                ma25_89_signal.append(0)

    def ma21_89_signal_func():
        global ma21_89_signal
        signal=0
        ma21_89_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA21'][i] > dfa1['MA89'][i]:
                if signal != 1:
                    signal = 1
                    ma21_89_signal.append(signal)
                else:
                    ma21_89_signal.append(0)
            elif dfa1['MA21'][i] < dfa1['MA89'][i]:
                if signal != -1:
                    signal = -1
                    ma21_89_signal.append(signal)
                else:
                    ma21_89_signal.append(0)
            else:
                ma21_89_signal.append(0)
    def ma30_200_signal_func():
        global ma30_200_signal
        ma30_200_signal = []
        signal = 0
        for i in range(len(dfa1)):
            if dfa1['MA30'][i] > dfa1['MA200'][i]:
                if signal != 1:
                    signal = 1
                    ma30_200_signal.append(signal)
                else:
                    ma30_200_signal.append(0)
            elif dfa1['MA30'][i] < dfa1['MA200'][i]:
                if signal != -1:
                    signal = -1
                    ma30_200_signal.append(signal)
                else:
                    ma30_200_signal.append(0)
            else:
                ma30_200_signal.append(0)

    def ma5_10_signal_func():
        global ma5_10_signal
        signal = 0
        ma5_10_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA5'][i] > dfa1['MA10'][i]:
                if signal != 1:
                    signal = 1
                    ma5_10_signal.append(signal)
                else:
                    ma5_10_signal.append(0)
            elif dfa1['MA5'][i] < dfa1['MA10'][i]:
                if signal != -1:
                    signal = -1
                    ma5_10_signal.append(signal)
                else:
                    ma5_10_signal.append(0)
            else:
                ma5_10_signal.append(0)

    def ma50_200_signal_func():
        global ma50_200_signal
        ma50_200_signal=[]
        signal = 0
        for i in range(len(dfa1)):
            if dfa1['MA50'][i] > dfa1['MA200'][i]:
                if signal != 1:
                    signal = 1
                    ma50_200_signal.append(signal)
                else:
                    ma50_200_signal.append(0)
            elif dfa1['MA50'][i] < dfa1['MA200'][i]:
                if signal != -1:
                    signal = -1
                    ma50_200_signal.append(signal)
                else:
                    ma50_200_signal.append(0)
            else:
                ma50_200_signal.append(0)

    def ma50_100_signal_func():
        global ma50_100_signal
        ma50_100_signal=[]
        signal=0
        for i in range(len(dfa1)):
            if dfa1['MA50'][i] > dfa1['MA100'][i]:
                if signal != 1:
                    signal = 1
                    ma50_100_signal.append(signal)
                else:
                    ma50_100_signal.append(0)
            elif dfa1['MA50'][i] < dfa1['MA100'][i]:
                if signal != -1:
                    signal = -1
                    ma50_100_signal.append(signal)
                else:
                    ma50_100_signal.append(0)
            else:
                ma50_100_signal.append(0)

    def ma30_50_signal_func():
        global ma30_50_signal
        ma30_50_signal = []
        signal = 0
        for i in range(len(dfa1)):
            if dfa1['MA30'][i] > dfa1['MA50'][i]:
                if signal != 1:
                    signal = 1
                    ma30_50_signal.append(signal)
                else:
                    ma30_50_signal.append(0)
            elif dfa1['MA30'][i] < dfa1['MA50'][i]:
                if signal != -1:
                    signal = -1
                    ma30_50_signal.append(signal)
                else:
                    ma30_50_signal.append(0)
            else:
                ma30_50_signal.append(0)

    def obv_signal_func():
        global obv_signal
        obv_signal = []
        signal = 0
        for i in range(len(dfa1)):
            if obv[i] > obv_mean[i]:
                if signal != 1:
                    signal = 1
                    obv_signal.append(signal)
                else:
                    obv_signal.append(0)
            elif obv[i] < obv_mean[i]:
                if signal != -1:
                    signal = -1
                    obv_signal.append(signal)
                else:
                    obv_signal.append(0)
            else:
                obv_signal.append(0)

    def obv_rsi_signal_changes_func():
        global obv_rsi_signal,obv
        obv_rsi_signal = []
        signal = 0
        for i in range(len(prices)):
            if (obv[i]>=obv[i-1]*1.04 and RSI[i]<35):
                obv_rsi_signal.append(1)
            elif (obv[i]<=obv[i-1]*0.96) and RSI[i]>70:
                obv_rsi_signal.append(-1)
            else:
                obv_rsi_signal.append(0)

    def obv_signal_changes_func():
        global obv_signal_changes_signal,obv
        obv_signal_changes_signal = []
        signal = 0
        for i in range(len(prices)):
            if (obv[i]>=obv[i-1]*1.1):
                obv_signal_changes_signal.append(1)
            elif (obv[i]<=obv[i-1]*0.9):
                obv_signal_changes_signal.append(-1)
            else:
                obv_signal_changes_signal.append(0)
    def obv_ma30_signal_changes_func():
        global obv_ma30_signal,obv
        obv_ma30_signal = []
        signal = 0
        for i in range(len(prices)):
            if (obv[i]>=obv[i-1]*1.02 and dfa1['MA30'][i]>prices[i]):
                obv_ma30_signal.append(1)
            elif (obv[i]<=obv[i-1]*0.98 and dfa1['MA30'][i]<prices[i]):
                obv_ma30_signal.append(-1)
            else:
                obv_ma30_signal.append(0)
    obv_ma30_signal_changes_func()


    def sma100_signal_func():
        global sma100_signal,prices,MA100
        MA100=ta.MA(dfa1_all['Close'], timeperiod=100, matype=0)
        MA100=MA100.loc[Sdate:Edate]
        MA100=MA100.array
        dfa1['MA100']=MA100
        prices=dfa1['Close']
        signal = 0
        sma100_signal=[]
        for i in range(len(prices)):
            if i==0 :
                signal=0
            else:
                if prices[i-1]<=MA100[i] and prices[i]>=MA100[i]:
                    signal = 1
                elif prices[i-1]>=MA100[i] and prices[i]<=MA100[i]:
                    signal=-1
                else:
                    signal=0
            sma100_signal.append(signal)

    def sma89_signal_func():
        global sma89_signal
        MA89=ta.MA(dfa1_all['Close'], timeperiod=89, matype=0)
        MA89=MA89.loc[Sdate:Edate]
        MA89=MA89.array
        dfa1['MA89']=MA89
        prices=dfa1['Close']
        signal = 0
        sma89_signal=[]
        for i in range(len(prices)):
            if (i==0):
                signal=0
            else:
                if prices[i-1]<=MA89[i] and prices[i]>=MA89[i]:
                    signal = 1
                elif prices[i-1]>=MA89[i] and prices[i]<=MA89[i]:
                    signal=-1
                else:
                    signal=0
            sma89_signal.append(signal)

    def sma25_signal_func():
        global sma25_signal
        MA25=ta.MA(dfa1_all['Close'], timeperiod=25, matype=0)
        MA25=MA25.loc[Sdate:Edate]
        MA25=MA25.array
        dfa1['MA25']=MA25
        prices=dfa1['Close']
        signal = 0
        sma25_signal=[]
        for i in range(len(prices)):
            if (i==0):
                signal=0
            else:
                if prices[i-1]<=MA25[i] and prices[i]>=MA25[i]:
                    signal = 1
                elif prices[i-1]>=MA25[i] and prices[i]<=MA25[i]:
                    signal=-1
                else:
                    signal=0
            sma25_signal.append(signal)

    def ma30_100_signal_func():
        global ma30_100_signal
        signal = 0
        ma30_100_signal=[]
        for i in range(len(dfa1)):
            if dfa1['MA30'][i] > dfa1['MA100'][i]:
                if signal != 1:
                    signal = 1
                    ma30_100_signal.append(signal)
                else:
                    ma30_100_signal.append(0)
            elif dfa1['MA30'][i] < dfa1['MA100'][i]:
                if signal != -1:
                    signal = -1
                    ma30_100_signal.append(signal)
                else:
                    ma30_100_signal.append(0)
            else:
                ma30_100_signal.append(0)

    def macd_crossover_signal_func():
        global macd_crossover_signal
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        dfa1['macdsignal']=macdsignal
        dfa1['macdhist']=macdhist
        data_macd=dfa1
        data=dfa1
        prices=dfa1['Open']
        macd_crossover_signal = []
        signal = 0
        for i in range(len(data)):
            if data_macd['MACD'][i] > data_macd['macdsignal'][i]:
                if signal != 1:
                    signal = 1
                    macd_crossover_signal.append(signal)
                else:
                    macd_crossover_signal.append(0)
            elif data_macd['MACD'][i] < data_macd['macdsignal'][i]:
                if signal != -1:
                    signal = -1
                    macd_crossover_signal.append(signal)
                else:
                    macd_crossover_signal.append(0)
            else:
                macd_crossover_signal.append(0)

    def macd_overzero_signal_func():
        global macd_overzero_signal
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        dfa1['macdsignal']=macdsignal
        dfa1['macdhist']=macdhist
        data_macd=dfa1
        data=dfa1
        prices=dfa1['Open']
        macd_overzero_signal=[]
        for i in range(len(data)):
            if data_macd['MACD'][i] > 0:
                macd_overzero_signal.append(1)
            else:
                macd_overzero_signal.append(-1)

    def macd_kongpop_signal_func():
        global macd_kongpop_signal
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        dfa1['macdsignal']=macdsignal
        dfa1['macdhist']=macdhist
        data_macd=dfa1
        data=dfa1
        prices=dfa1['Open']
        macd_kongpop_signal=[]
        for i in range(len(data)):
            if i==0:
                pass
            if data_macd['MACD'][i-1] <= -0.3 and data_macd['MACD'][i-1] >=-0.3:
                macd_kongpop_signal.append(1)
            elif data_macd['MACD'][i-1] >= 0.5 and data_macd['MACD'][i-1] <=0.5:
                macd_kongpop_signal.append(-1)
            else:
                macd_kongpop_signal.append(0)

    def adx_signal_func():
        global adx_signal
        adx_signal = []
        signal = 0
        for i in range(len(prices)):
            if adx[i-1] < 25 and adx[i] > 25 and pdi[i] > ndi[i]:
                if signal != 1:
                    signal = 1
                    adx_signal.append(signal)
                else:
                    adx_signal.append(0)
            elif adx[i-1] < 25 and adx[i] > 25 and ndi[i] > pdi[i]:
                if signal != -1:
                    signal = -1
                    adx_signal.append(signal)
                else:
                    adx_signal.append(0)
            else:
                adx_signal.append(0)

    fastk, fastd = ta.STOCH(dfa1_all['High'], dfa1_all['Low'], dfa1_all['Close'],14,3,False)
    K=fastk.loc[Sdate:Edate]
    K=K.array
    dfa1['%K']=K
    D=fastd.loc[Sdate:Edate]
    D=D.array
    dfa1['%D']=D
    k=dfa1['%K']
    d=dfa1['%D']

    def stoch_signal_func():
        global stoch_signal
        stoch_signal = []
        signal = 0
        for i in range(len(dfa1['Close'])):
            k=dfa1['%K']
            d=dfa1['%D']
            if k[i] < 20 and d[i] < 20 and k[i] < d[i]:
                if signal != 1:
                    signal = 1
                    stoch_signal.append(signal)
                else:
                    stoch_signal.append(0)
            elif k[i] > 80 and d[i] > 80 and k[i] > d[i]:
                if signal != -1:
                    signal = -1
                    stoch_signal.append(signal)
                else:
                    stoch_signal.append(0)
            else:
                stoch_signal.append(0)

    def adx_sto_kp_signal_func():
        global adx_sto_kp_signal
        adx_sto_kp_signal = []
        signal = 0
        zone_sto = 0
        for i in range(len(prices)):
            if d[i]>=80:
                zone_sto=1
            else:
                zone_sto=0
            if adx[i]>25 and d[i-1]<=20 and d[i]>=20:
                adx_sto_kp_signal.append(1)
            elif d[i]<d[i-1] and zone_sto==1 and adx[i]>=25:
                adx_sto_kp_signal.append(-1)
            else:
                adx_sto_kp_signal.append(0)

    def stoch_signal_2_func():
        global stoch_signal_2
        stoch_signal_2=[]
        for i in range(len(dfa1)):
            if k[i] > d[i] and k[i-1] < d[i-1] and k[i]<=20:
                stoch_signal_2.append(1)
            elif k[i] < d[i] and k[i-1] > d[i-1] and k[i]>=80:
                stoch_signal_2.append(-1)
            else:
                stoch_signal_2.append(0)
    global atr
    high_low = dfa1_all['High'] - dfa1_all['Low']
    high_close = np.abs(dfa1_all['High'] - dfa1_all['Close'].shift())
    low_close = np.abs(dfa1_all['Low'] - dfa1_all['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    atr = true_range.rolling(14).sum()/14
    atr=atr.loc[Sdate:Edate]

    def roc_signal_func():
        global roc_signal
        roc = ta.ROC(dfa1_all["Close"], timeperiod=10)
        roc=roc.loc[Sdate:Edate]
        data=roc
        roc_signal = []
        signal = 0
        for i in range(len(data)):
            if data[i] > 0:
                if signal != 1:
                    signal = 1
                    roc_signal.append(signal)
                else:
                    roc_signal.append(0)
            elif data[i] < 0:
                if signal != -1:
                    signal = -1
                    roc_signal.append(signal)
                else:
                    roc_signal.append(0)
            else:
                roc_signal.append(0)

    RSI = ta.RSI(dfa1_all['Close'], timeperiod=14)
    RSI=RSI.loc[Sdate:Edate]
    RSI=RSI.array

    def rsi_signal_func():
        global rsi_signal
        rsi_signal = []
        signal = 0
        for i in range(len(prices)):
            if RSI[i] <= 35:
                signal=1
            elif RSI[i] >= 75:
                signal=-1
            else:
                signal=0
            rsi_signal.append(signal)

    def implement_adx_signal_func():
        global implement_adx_signal
        implement_adx_signal = []
        signal = 0
        for i in range(len(prices)):
            if ADX[i-1] < 25 and ADX[i] > 25 and pdi[i] > ndi[i]:
                if signal != 1:
                    signal = 1
                    implement_adx_signal.append(signal)
                else:
                    implement_adx_signal.append(0)
            elif ADX[i-1] < 25 and ADX[i] > 25 and ndi[i] > pdi[i]:
                if signal != -1:
                    signal = -1
                    implement_adx_signal.append(signal)
                else:
                    implement_adx_signal.append(0)
            else:
                implement_adx_signal.append(0)

    def rsi_adx_signal_func():
        global rsi_adx_signal
        RSI=dfa1['RSI']
        rsi_adx_signal = []
        signal = 0
        for i in range(len(RSI)):
            if RSI[i] <= 30 and RSI[i]>RSI[i-1] and ADX[i]>=35:
                rsi_adx_signal.append(1)
            elif RSI[i] >= 70 and RSI[i]<RSI[i-1] and ADX[i]>=35:
                rsi_adx_signal.append(-1)
            else:
                rsi_adx_signal.append(0)

    def cci_signal_func():
        global cci_signal
        CCI=ta.CCI(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        CCI=CCI.loc[Sdate:Edate]
        cci_signal = []
        data=CCI
        signal = 0
        countb,counts =0,0
        for i in range(len(data)):
            if data[i] <= -90:
                counts = 0
                countb = countb+1
                if countb == 6 and signal != 1 :
                    signal = 1
                    cci_signal.append(signal)
                else:
                    cci_signal.append(0)
            elif data[i] >90:
                countb = 0
                counts = counts+1
                if counts == 6 and signal != -1:
                    signal = -1
                    cci_signal.append(signal)
                else:
                    cci_signal.append(0)
            else:
                cci_signal.append(0)
                countb,counts =0,0

    def cci_kp_signal_func():
        global cci_kp_signal
        CCI=ta.CCI(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        CCI=CCI.loc[Sdate:Edate]
        cci_kp_signal = []
        data=CCI
        signal = 0
        for i in range(len(data)):
            if (i==0):
                signal=0
            else:
                if data[i] <= -180 and data[i]>=data[i-1] and data[i-2]>=data[i-1]:
                    signal=1
                elif data[i] >=180 and data[i]<=data[i-1] and data[i-2]<=data[i-1]:
                    signal=-1
                else:
                    signal=0
            cci_kp_signal.append(signal)

    def cci_adx_signal_func():
        global cci_adx_signal
        cci_adx_signal=[]
        signal = 0
        CCI=ta.CCI(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        CCI=CCI.loc[Sdate:Edate]
        data=CCI
        for i in range(len(data)):
            if (data[i] <= -100 and data[i]>=data[i-1] and data[i-2]>=data[i-1] and ADX[i]>25):
                signal = 1
            elif (data[i] >= 100 and ADX[i]>40 and ADX[i]<ADX[i-1]):
                signal = -1
            else:
                signal=0
            cci_adx_signal.append(signal)
    #cum MA5
    def cci_ma5_signal_func():
        global cci_ma5_signal
        cci_ma5_signal=[]
        signal = 0
        CCI=ta.CCI(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        CCI=CCI.loc[Sdate:Edate]
        data=CCI
        for i in range(len(data)):
            if (data[i] <= -150 and MA5[i-1]>dfa1['Close'][i-1] and MA5[i]<dfa1['Close'][i]):
                signal = 1
            elif (data[i] >= 150 and MA5[i-1]<dfa1['Close'][i-1] and MA5[i]>dfa1['Close'][i]):
                signal = -1
            else:
                signal=0
            cci_ma5_signal.append(signal)

    def atr_signal_func():
        global atr_signal
        atr_signal=[0]*len(dfa1)

    def wpr_signal_func():
        global wpr_signal
        wpr=ta.WILLR(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        wpr=wpr.loc[Sdate:Edate]
        wpr_signal = []
        wr=wpr
        signal = 0
        for i in range(len(wr)):
            if wr[i] < -80 and wr[i]>wr[i-1]:
                if signal != 1:
                    signal = 1
                    wpr_signal.append(signal)
                else:
                    wpr_signal.append(0)
            elif wr[i] > -20 and wr[i]<wr[i-1]:
                if signal != -1:
                    signal = -1
                    wpr_signal.append(signal)
                else:
                    wpr_signal.append(0)
            else:
                wpr_signal.append(0)

    def bb_signal_func():
        global bb_signal
        upperband, middleband, lowerband = ta.BBANDS(dfa1_all['Close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
        upperband, middleband, lowerband=upperband, middleband, lowerband.loc[Sdate:Edate]
        upperband, middleband, lowerband=upperband, middleband, lowerband.array
        dfa1['upperband']=upperband
        dfa1['middleband']=middleband
        dfa1['lowerband']=lowerband
        buy_price = []
        sell_price = []
        bb_signal = []
        signal = 0
        data=dfa1['Open']
        lower_bb=dfa1['lowerband']
        upper_bb=dfa1['upperband']
        a,b,c=dfa1['Close'], dfa1['lowerband'], dfa1['upperband']
        for i in range(len(data)):
            if data[i-1] > lower_bb[i-1] and data[i] < lower_bb[i]:
                if signal != 1:
                    signal = 1
                    bb_signal.append(signal)
                else:
                    bb_signal.append(0)
            elif data[i-1] < upper_bb[i-1] and data[i] > upper_bb[i]:
                if signal != -1:
                    signal = -1
                    bb_signal.append(signal)
                else:
                    bb_signal.append(0)
            else:
                bb_signal.append(0)

    def cdc_signal_func():
        global cdc_signal
        ohlc4=(dfa1_all['Close']+dfa1_all['Low']+dfa1_all['High']+dfa1_all['Open'])/4
        EMA2=ta.EMA(ohlc4, timeperiod=2)
        EMA26=ta.EMA(EMA2, timeperiod=26)
        EMA12=ta.EMA(EMA2, timeperiod=12)
        EMA26=EMA26.loc[Sdate:Edate]
        EMA26=EMA26.array
        dfa1['EMA26']=EMA26

        EMA12=EMA12.loc[Sdate:Edate]
        EMA12=EMA12.array
        dfa1['EMA12']=EMA12

        EMA2=EMA2.loc[Sdate:Edate]
        EMA2=EMA2.array
        dfa1['EMA2']=EMA2

        ohlc4=ohlc4.loc[Sdate:Edate]
        ohlc4=ohlc4.array
        dfa1['ohlc4']=ohlc4
        fastslow = []
        for i in range(len(dfa1)):
            if dfa1['EMA12'][i] > dfa1['EMA26'][i]:
                fastslow.append(1)
            elif dfa1['EMA26'][i] > dfa1['EMA12'][i]:
                fastslow.append(-1)
            else:
                fastslow.append(0)

        color = []
        for i in range(len(dfa1)):
            if fastslow[i] == 1 and dfa1['EMA2'][i] > dfa1['EMA12'][i]:
                color.append(1)
            elif fastslow[i] == -1 and dfa1['EMA2'][i] > dfa1['EMA12'][i]:
                color.append(0.5)
            elif fastslow[i] == -1 and dfa1['EMA2'][i] < dfa1['EMA12'][i]:
                color.append(-1)
            elif fastslow[i] == 1 and dfa1['EMA2'][i] < dfa1['EMA12'][i]:
                color.append(-0.5)
            else:
                color.append(0)
        signal=0
        cdc_signal=[]
        for i in range (len(color)):
            if color[i] == 1:
                if signal != 1:
                    signal = 1
                    cdc_signal.append(signal)
                else:
                    cdc_signal.append(0)
            elif color[i] == -1:
                if signal != -1:
                    signal = -1
                    cdc_signal.append(signal)
                else:
                    cdc_signal.append(0)
            elif color[i] == 0.5:
                if signal != 0.5:
                    signal = 0.5
                    cdc_signal.append(signal)
                else:
                    cdc_signal.append(0)
            elif color[i] == -0.5:
                if signal != -0.5:
                    signal = -0.5
                    cdc_signal.append(signal)
                else:
                    cdc_signal.append(0)
            else:
                signal=0
                cdc_signal.append(signal)

    def sar_signal_func():
        global sar_signal
        SAR=ta.SAR(dfa1_all['High'], dfa1_all['Low'], acceleration=0.02, maximum=0.2)
        SAR=SAR.loc[Sdate:Edate]
        SAR=SAR.array
        dfa1['SAR']=SAR
        signal= 0
        countb,counts =0,0
        sar_signal = []
        for i in range(len(dfa1)):
            if dfa1['Close'][i] > dfa1['SAR'][i]:
                counts = 0
                countb =countb +1
                if countb == 2:
                    signal = 1
                    sar_signal.append(signal)
                else:
                    sar_signal.append(0)
            elif dfa1['Close'][i] < dfa1['SAR'][i]:
                countb = 0
                counts =counts +1
                if counts == 2:
                    signal = -1
                    sar_signal.append(signal)
                else:
                    sar_signal.append(0)
            else:
                sar_signal.append(0)

    def aroon_signal_func():
        global aroon_signal
        down,up=ta.AROON(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
        down=down.loc[Sdate:Edate]
        up=up.loc[Sdate:Edate]
        down=down.array
        up=up.array
        aroon_signal = []
        signal = 0
        for i in range(len(dfa1)):
            if up[i] >= 70 and down[i] <= 30:
                if signal != 1:
                    signal = 1
                    aroon_signal.append(signal)
                else:
                    aroon_signal.append(0)
            elif up[i] <= 30 and down[i] >= 70:
                if signal != -1:
                    signal = -1
                    aroon_signal.append(signal)
                else:
                    aroon_signal.append(0)
            else:
                aroon_signal.append(0)

    def rsiandmacd_signal_func():
        global rsiandmacd_signal,itt
        itt=prices
        rsiandmacd_signal=[]
        for i in range(len(dfa1)):
            if i!=0:
                if prices[i] <= 35 and macd_crossover_signal[i]==1 and prices[i]>prices[i-1]:
                    rsiandmacd_signal.append(1)
                elif prices[i] >= 45 and macd_crossover_signal[i]==-1:
                    rsiandmacd_signal.append(-1)
                else:
                    rsiandmacd_signal.append(0)
            else:
                    rsiandmacd_signal.append(0)
    '''def rsiandmacd_signal_func():
        global rsiandmacd_signal
        rsiandmacd_signal=[]
        for i in range(len(dfa1)):
            if i!=0:
                if prices[i] <= 35 and macd_crossover_signal[i]==1 and prices[i]>prices[i-1]:
                    rsiandmacd_signal.append(1)
                elif prices[i] >= 65 and macd_crossover_signal[i]==-1 and prices[i]<=prices[i-1]:
                    rsiandmacd_signal.append(-1)
                else:
                    rsiandmacd_signal.append(0)
            else:
                    rsiandmacd_signal.append(0)'''
    def renko_signal_func():
        global renko_signal
        renko_signal=[]
        renko_brick=[]
        renko_start=dfa1['Close'][0]
        for i in range(len(dfa1)):
            if i!=0:
                if dfa1['Close'][i]>=1.01*renko_start:
                    renko_brick.append(1)
                    renko_start=dfa1['Close'][i]
                elif dfa1['Close'][i]<=0.99*renko_start:
                    renko_brick.append(-1)
                    renko_start=dfa1['Close'][i]
                else:
                    renko_brick.append(0)
                if renko_brick[i]==1 and renko_brick[i-1]==1:
                    renko_signal.append(1)
                elif renko_brick[i]==-1 and renko_brick[i-1]==-1:
                    renko_signal.append(-1)
                else:
                    renko_signal.append(0)
            else:
                renko_brick.append(0)
                renko_signal.append(0)

    def renko_macd_crossover_signal_func():
        global renko_macd_crossover_signal
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        data_macd=dfa1
        data=dfa1
        prices=dfa1['Open']
        renko_macd_crossover_signal=[]
        for i in range(len(dfa1)):
            if renko_signal[i]==1 and data_macd['MACD'][i-1] < data_macd['macdsignal'][i-1] and data_macd['MACD'][i] > data_macd['macdsignal'][i]:
                renko_macd_crossover_signal.append(1)
            elif renko_signal[i]==-1 and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i] < data_macd['macdsignal'][i]:
                renko_macd_crossover_signal.append(-1)
            else:
                renko_macd_crossover_signal.append(0)

    def renko_macdzero_signal_func():
        global renko_macdzero_signal
        renko_macdzero_signal=[]
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        data_macd=dfa1
        for i in range(len(dfa1)):
            if i>=4:
                if renko_signal[i]==1 and data_macd['MACD'][i] > 0:
                    renko_macdzero_signal.append(1)
                elif renko_signal[i]==-1 and data_macd['MACD'][i] < 0:
                    renko_macdzero_signal.append(-1)
                else:
                    renko_macdzero_signal.append(0)
            else:
                renko_macdzero_signal.append(0)
    def renko_macdindian_signal_func():
        global renko_macdindian_signal
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        data_macd=dfa1
        data=dfa1
        prices=dfa1['Open']
        renko_macdindian_signal=[]
        for i in range(len(dfa1)):
            if i>=4:
                if renko_signal[i]==1 and data_macd['MACD'][i] > data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] > data_macd['macdsignal'][i-2] and data_macd['MACD'][i-3] > data_macd['macdsignal'][i-3] and data_macd['MACD'][i-4] > data_macd['macdsignal'][i-4]:
                    renko_macdindian_signal.append(1)
                elif renko_signal[i]==-1 and data_macd['MACD'][i] < data_macd['macdsignal'][i] and data_macd['MACD'][i-1] < data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] < data_macd['macdsignal'][i-2]and data_macd['MACD'][i-3] < data_macd['macdsignal'][i-3]and data_macd['MACD'][i-4] < data_macd['macdsignal'][i-4]:
                    renko_macdindian_signal.append(-1)
                else:
                    renko_macdindian_signal.append(0)
            else:
                renko_macdindian_signal.append(0)

    def fake_obv_renko_signal_func():
        global fake_obv_renko_signal,fake_price
        fake_obv_renko_signal=[]
        fake_price=ta.LINEARREG_SLOPE(dfa1_all['Close'],timeperiod=14)
        fake_price=fake_price.loc[Sdate:Edate]
        for i in range(len(dfa1)):
            if fake_price[i]<=0.3 and renko_signal[i]==1:
                fake_obv_renko_signal.append(1)
            elif fake_price[i]>0.3 and renko_signal[i]==-1:
                fake_obv_renko_signal.append(-1)
            else:
                fake_obv_renko_signal.append(0)

    def fake_obv_macd_crossover_signal_func():
        global fake_obv_macd_crossover_signal
        fake_obv_macd_crossover_signal=[]
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        data_macd=dfa1
        fake_price=ta.LINEARREG_SLOPE(dfa1_all['Close'],timeperiod=14)
        fake_price=fake_price.loc[Sdate:Edate]
        for i in range(len(dfa1)):
            if fake_price[i]<=0.3 and data_macd['MACD'][i] > data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] > data_macd['macdsignal'][i-2] and data_macd['MACD'][i-3] > data_macd['macdsignal'][i-3] and data_macd['MACD'][i-4] > data_macd['macdsignal'][i-4]:
                fake_obv_macd_crossover_signal.append(1)
            elif fake_price[i]>0.3 and data_macd['MACD'][i] < data_macd['macdsignal'][i] and data_macd['MACD'][i-1] < data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] < data_macd['macdsignal'][i-2]and data_macd['MACD'][i-3] < data_macd['macdsignal'][i-3]and data_macd['MACD'][i-4] < data_macd['macdsignal'][i-4]:
                fake_obv_macd_crossover_signal.append(-1)
            else:
                fake_obv_macd_crossover_signal.append(0)

    def real_obv_macd_crossover_signal_func():
        global real_obv_macd_crossover_signal,real_price
        macd, macdsignal, macdhist = ta.MACD(dfa1_all['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        MACD=macd.loc[Sdate:Edate]
        dfa1['MACD']=MACD
        data_macd=dfa1
        real_price=ta.LINEARREG_SLOPE(obv,timeperiod=14)
        real_price=real_price.loc[Sdate:Edate]
        real_obv_macd_crossover_signal=[]
        for i in range(len(dfa1)):
            if real_price[i]>=0.3 and data_macd['MACD'][i] > data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] and data_macd['MACD'][i-2] > data_macd['macdsignal'][i-2] and data_macd['MACD'][i-3] > data_macd['macdsignal'][i-3] and data_macd['MACD'][i-4] > data_macd['macdsignal'][i-4]:
                real_obv_macd_crossover_signal.append(1)
            elif data_macd['MACD'][i] < data_macd['macdsignal'][i] and data_macd['MACD'][i-1] > data_macd['macdsignal'][i-1] :
                real_obv_macd_crossover_signal.append(-1)
            else:
                real_obv_macd_crossover_signal.append(0)

    def stoch2_adx_signal_func():
        global stoch2_adx_signal
        stoch2_adx_signal=[]
        for i in range(len(dfa1)):
            if stoch_signal_2[i]==1 and ADX[i]>30:
                stoch2_adx_signal.append(1)
            elif stoch_signal_2[i]==-1 and ADX[i]>35 and ADX[i]<ADX[i-1] :
                stoch2_adx_signal.append(-1)
            else:
                stoch2_adx_signal.append(0)

    def stoch1_adx_signal_func():
        global stoch1_adx_signal
        stoch1_adx_signal=[]
        for i in range(len(dfa1)):
            if k[i-1]<d[i-1] and k[i]>d[i] and k[i]>20 and d[i]>20 and ADX[i]>20:
                stoch1_adx_signal.append(1)
            elif k[i-1]>d[i-1] and k[i]<d[i] and ADX[i]>30 and ADX[i]<ADX[i-1] :
                stoch1_adx_signal.append(-1)
            else:
                stoch1_adx_signal.append(0)
    '''
    def stoch1_adx_signal_func():
        global stoch1_adx_signal
        stoch1_adx_signal=[]
        for i in range(len(dfa1)):
            if k[i-1]<d[i-1] and k[i]>d[i] and k[i]>20 and d[i]>20 and ADX[i]>30 and ADX[i]<ADX[i-1]:
                stoch1_adx_signal.append(1)
            elif k[i-1]>d[i-1] and k[i]<d[i] and ADX[i]>30 and ADX[i]<ADX[i-1] :
                stoch1_adx_signal.append(-1)
            else:
                stoch1_adx_signal.append(0)
    '''

    def bb_rsi_signal_func():
        global bb_rsi_signal
        bb_rsi_signal=[]
        for i in range(len(data)):
            if (bb_signal[i]==1 and RSI[i]>=50):
                bb_rsi_signal.append(1)
            elif (bb_signal[i]==-1 and RSI[i]<=40):
                bb_rsi_signal.append(-1)
            else:
                bb_rsi_signal.append(0)

    def psar_wpr_signal_func():
        global psar_wpr_signal
        psar_wpr_signal=[]
        for i in range(len(data)):
            if (psar_changetrend_signal[i]==1 and wpr_signal[i]!=1):
                psar_wpr_signal.append(1)
            elif (psar_changetrend_signal[i]==-1 and wpr_signal[i]!=-1):
                psar_wpr_signal.append(-1)
            else:
                psar_wpr_signal.append(0)

    def cdc_atr_signal_func():
        global cdc_atr_signal
        cdc_atr_signal=[]
        for i in range(len(data)):
            if (cdc_signal[i]==1 and atr_signal[i]==1):
                cdc_atr_signal.append(1)
            elif (cdc_signal[i]==-1 and atr_signal[i]==1):
                cdc_atr_signal.append(-1)
            else:
                cdc_atr_signal.append(0)

    def macd_sto_signal_func():
        global macd_sto_signal
        macd_sto_signal=[]
        for i in range(len(data)):
            if (stoch_signal[i]==1 and macd_crossover_signal[i]==1):
                macd_sto_signal.append(1)
            elif (stoch_signal[i]==-1 and macd_crossover_signal[i]==-1):
                macd_sto_signal.append(-1)
            else:
                macd_sto_signal.append(0)

    def sto_ma50_200_signal_func():
        global sto_ma50_200_signal
        sto_ma50_200_signal=[]
        for i in range(len(data)):
            if (stoch_signal[i]==1 and ma50_200_signal[i]==1):
                sto_ma50_200_signal.append(1)
            elif (stoch_signal[i]==-1 and ma50_200_signal[i]==-1):
                sto_ma50_200_signal.append(-1)
            else:
                sto_ma50_200_signal.append(0)

    def atr_adx_signal_func():
        global atr_adx_signal
        atr_adx_signal=[]
        for i in range(len(data)):
            if (atr_signal[i]==1 and adx_signal[i]==1):
                atr_adx_signal.append(1)
            elif (atr_signal[i]==-1 and adx_signal[i]==-1):
                atr_adx_signal.append(-1)
            else:
                atr_adx_signal.append(0)

    def rsi_divergent_signal_func():
        global rsi_divergent_signal
        data=rsiuse(dfa1)
        rsi_divergent_signal=rsidiver()

    def psar_adx_signal_func():
        global psar_adx_signal
        psar_adx_signal=[]
        for i in range(len(data)):
            if (psar_trend_signal[i]==1 and ADX[i]>20 and plus_DMI[i] < minus_DMI[i]):
                psar_adx_signal.append(1)
            elif (psar_trend_signal[i]==-1 and ADX[i]>40 and plus_DMI[i] > minus_DMI[i]):
                psar_adx_signal.append(-1)
            else:
                psar_adx_signal.append(0)

    def cci_renko_signal_func():
        global cci_renko_signal
        cci_renko_signal=[]
        for i in range(len(data)):
            if (renko_signal[i]==1 and RSI[i] <= 35):
                cci_renko_signal.append(1)
            elif (renko_signal[i]==-1 and RSI[i] >= 65):
                cci_renko_signal.append(-1)
            else:
                cci_renko_signal.append(0)

    def macd_crossover_wpr_signal_func():
        global macd_crossover_wpr_signal
        macd_crossover_wpr_signal=[]
        wpr=ta.WILLR(dfa1_all["High"],dfa1_all["Low"],dfa1_all["Close"],timeperiod=14)
        wpr=wpr.loc[Sdate:Edate]
        for i in range(len(data)):
            if (macd_crossover_signal[i]==1 and wpr[i] <= -80):
                macd_crossover_wpr_signal.append(1)
            elif (macd_crossover_signal[i]==-1 and wpr[i] <= -20):
                macd_crossover_wpr_signal.append(-1)
            else:
                macd_crossover_wpr_signal.append(0)

    def aroon_kp_signal_func():
        global aroon_kp_signal
        aroon_kp_signal=[]
        down,up=ta.AROON(dfa1_all['High'], dfa1_all['Low'], timeperiod=14)
        down=down.loc[Sdate:Edate]
        up=up.loc[Sdate:Edate]
        down=down.array
        for i in range(len(data)):
            if (up[i-1]<down[i-1] and up[i]>down[i]):
                aroon_kp_signal.append(1)
            elif (up[i-1]>down[i-1] and up[i]<down[i]):
                aroon_kp_signal.append(-1)
            else:
                aroon_kp_signal.append(0)

    def renko_adx_signal_func():
        global renko_adx_signal
        renko_adx_signal=[]
        for i in range(len(data)):
            if (renko_signal[i]==1 and ADX[i]>25):
                renko_adx_signal.append(1)
            elif (renko_signal[i]==-1 and ADX[i]<ADX[i-1] and ADX[i]>40):
                renko_adx_signal.append(-1)
            else:
                renko_adx_signal.append(0)

    def volume_prof_signal_func():
        global volume_prof_signal
        signal=0
        volume_prof_signal=[]
        data=dfa1
        volume = data['Volume']
        close = data['Close']
        kde_factor = 0.05
        num_samples = 500
        kde = stats.gaussian_kde(close,weights=volume,bw_method=kde_factor)
        xr = np.linspace(close.min(),close.max(),num_samples)
        kdy = kde(xr)
        ticks_per_sample = (xr.max() - xr.min()) / num_samples
        min_prom = 1
        min_prom = kdy.max() * 0.3
        peaks, peak_props = sg.find_peaks(kdy, prominence=min_prom)
        pkx = xr[peaks]
        pky = kdy[peaks]
        pkx=sorted(pkx,reverse=True)
        for i in range(len(data)):
            if len(pkx)==1:
                if close[i] > pkx[0]:
                    if signal != 1:
                        signal = 1
                        volume_prof_signal.append(signal)
                    else:
                        volume_prof_signal.append(0)
                elif close[i] < pkx[0]:
                    if signal != -1:
                        signal = -1
                        volume_prof_signal.append(signal)
                    else:
                        volume_prof_signal.append(0)
                else:
                    volume_prof_signal.append(0)
            else:
                max=pkx[0]
                min=pkx[1]
                if (close[i]<min):
                    volume_prof_signal.append(1)
                elif (close[i]>max):
                    volume_prof_signal.append(-1)
                else:
                    volume_prof_signal.append(0)
    def volume_prof_adx_signal_func():
        global volume_prof_adx_signal
        signal=0
        volume_prof_adx_signal=[]
        data=dfa1
        volume = data['Volume']
        close = data['Close']
        kde_factor = 0.05
        num_samples = 500
        kde = stats.gaussian_kde(close,weights=volume,bw_method=kde_factor)
        xr = np.linspace(close.min(),close.max(),num_samples)
        kdy = kde(xr)
        ticks_per_sample = (xr.max() - xr.min()) / num_samples
        min_prom = 1
        min_prom = kdy.max() * 0.3
        peaks, peak_props = sg.find_peaks(kdy, prominence=min_prom)
        pkx = xr[peaks]
        pky = kdy[peaks]
        pkx=sorted(pkx,reverse=True)
        for i in range(len(data)):
            if len(pkx)==1:
                if close[i] > pkx[0]:
                    if signal != 1:
                        signal = 1
                        volume_prof_adx_signal.append(signal)
                    else:
                        volume_prof_adx_signal.append(0)
                elif close[i] < pkx[0]:
                    if signal != -1 and ADX[i]>35:
                        signal = -1
                        volume_prof_adx_signal.append(signal)
                    else:
                        volume_prof_adx_signal.append(0)
                else:
                    volume_prof_adx_signal.append(0)
            else:
                max=pkx[0]
                min=pkx[1]
                if (close[i]<min):
                    volume_prof_adx_signal.append(1)
                elif (close[i]>max and ADX[i]>35):
                    volume_prof_adx_signal.append(-1)
                else:
                    volume_prof_adx_signal.append(0)

    def vwap_signal_func():
        global vwap_signal
        vwap_signal=[]
        df=dfa1
        v = df['Volume'].values
        tp = (df['Low'] + df['Close'] + df['High']).div(3).values
        df=df.assign(vwap=(tp * v).cumsum() / v.cumsum())
        vwap=df['vwap']
        signal=0
        for i in range(len(data)):
            if vwap[i]<dfa1['Close'][i]:
                if signal != 1:
                    signal = 1
                    vwap_signal.append(signal)
                else:
                    vwap_signal.append(0)
            elif vwap[i]>dfa1['Close'][i] :
                if signal != -1:
                    signal = -1
                    vwap_signal.append(signal)
                else:
                    vwap_signal.append(0)
            else:
                vwap_signal.append(0)

    def rsi_delta_func():
        global rsi_delta_signal,RSIDEL
        rsi_delta_signal=[]
        RSIDEL=[]
        for i in range(len(prices)):
            rsidel=100-(100/(1+RSI[i]))
            rsidel=rsidel-50
            rsidel=rsidel-RSI[i]
            RSIDEL.append(rsidel)
            if i>=2:
                if RSIDEL[i]<-40 and RSIDEL[i-1]>RSIDEL[i] and RSIDEL[i-2]>RSIDEL[i]:
                    rsi_delta_signal.append(1)
                elif RSIDEL[i]>40 and RSIDEL[i-1]<RSIDEL[i] and RSIDEL[i-2]<RSIDEL[i]:
                    rsi_delta_signal.append(-1)
                else:
                    rsi_delta_signal.append(0)
            else:
                rsi_delta_signal.append(0)

    def rsi_divergent_sto_signal_func():
        global rsi_divergent_sto_signal
        rsi_divergent_sto_signal=[]
        for i in range(len(data)):
            if rsi_divergent_signal[i]==1 and RSI[i]<35 and stoch_signal_2[i]==1:
                rsi_divergent_sto_signal.append(1)
            elif rsi_divergent_signal[i]==-1 and RSI[i]>65 and stoch_signal_2[i]==-1:
                rsi_divergent_sto_signal.append(-1)
            else:
                rsi_divergent_sto_signal.append(0)

    def rsi_divergent_adx_signal_func():
        global rsi_divergent_adx_signal
        rsi_divergent_adx_signal=[]
        for i in range(len(data)):
            if rsi_divergent_signal[i]==1 and RSI[i]<35 and ADX[i]>=25:
                rsi_divergent_adx_signal.append(1)
            elif rsi_divergent_signal[i]==-1 and RSI[i]>65 and ADX[i]>=40:
                rsi_divergent_adx_signal.append(-1)
            else:
                rsi_divergent_adx_signal.append(0)

    def atr_obv_signal_func():
        global atr_obv_signal
        atr_obv_signal=[]
        for i in range(len(data)):
          if atr[i]>=atr[i-1]*1.02 and obv[i]>=obv[i-1]*1.03:
            atr_obv_signal.append(1)
          elif atr[i]>=atr[i-1]*1.02 and obv[i]<=obv[i-1]*0.97:
            atr_obv_signal.append(-1)
          else:
            atr_obv_signal.append(0)

    def atr_rsi_signal_func1():
        global atr_rsi_signal1
        atr_rsi_signal1=[]
        for i in range(len(data)):
          if atr[i]>=atr[i-1]*1.02 and RSI[i] >= 60:
            atr_rsi_signal1.append(-1)
          elif atr[i]>=atr[i-1]*1.02 and RSI[i] <= 30:
            atr_rsi_signal1.append(1)
          else:
            atr_rsi_signal1.append(0)

    def atr_rsi_signal_func2():
        global atr_rsi_signal2
        atr_rsi_signal2=[]
        for i in range(len(data)):
          if RSI[i] >= 70 and ADX[i]>40 and RSI[i]<RSI[i-1]:
            atr_rsi_signal2.append(-1)
          elif atr[i]/prices[i]>=0.085 and RSI[i] <= 30:
            atr_rsi_signal2.append(1)
          else:
            atr_rsi_signal2.append(0)

    def atr_macd_crossover_signal_func():
        global atr_macd_crossover_signal
        prices=dfa1['Close']
        atr_macd_crossover_signal=[]
        for i in range(len(data)):
          if macd_crossover_signal[i]==1:
            atr_macd_crossover_signal.append(1)
          elif atr[i]/prices[i]>=0.1 and macd_crossover_signal[i]==-1:
            atr_macd_crossover_signal.append(-1)
          else:
            atr_macd_crossover_signal.append(0)

    def atr_ma25_signal_func():
        global atr_ma25_signal
        atr_ma25_signal=[]
        prices=dfa1['Close']
        for i in range(len(data)):
          if atr[i]>=atr[i-1]*1.02 and dfa1['MA25'][i]<prices[i]:
            atr_ma25_signal.append(1)
          elif atr[i]>=atr[i-1]*1.02 and dfa1['MA25'][i]>prices[i]:
            atr_ma25_signal.append(-1)
          else:
            atr_ma25_signal.append(0)


    adx_signal_func()
    macd_crossover_signal_func()
    ma30_200_signal_func()
    ma50_200_signal_func()
    ma30_50_signal_func()
    cdc_signal_func()
    sar_signal_func()
    roc_signal_func()
    obv_signal_func()
    rsi_signal_func()
    bb_signal_func()
    wpr_signal_func()
    stoch_signal_func()
    cci_signal_func()
    sma100_signal_func()
    atr_signal_func()
    aroon_signal_func()
    stoch_signal_2_func()
    renko_signal_func()
    renko_macd_crossover_signal_func()
    fake_obv_renko_signal_func()
    ma50_100_signal_func()
    sma89_signal_func()
    ma30_100_signal_func()
    stoch2_adx_signal_func()
    bb_rsi_signal_func()
    psar_wpr_signal_func()
    cdc_atr_signal_func()
    ma5_10_signal_func()
    macd_sto_signal_func()
    sto_ma50_200_signal_func()
    atr_adx_signal_func()
    ma25_89_signal_func()
    ma21_89_signal_func()
    macd_overzero_signal_func()
    macd_kongpop_signal_func()
    rsi_divergent_signal_func()
    adx_sto_kp_signal_func()
    real_obv_macd_crossover_signal_func()
    renko_macdzero_signal_func()
    rsi_adx_signal_func()
    rsiandmacd_signal_func()
    fake_obv_macd_crossover_signal_func()
    psar_adx_signal_func()
    cci_renko_signal_func()
    macd_crossover_wpr_signal_func()
    aroon_kp_signal_func()
    renko_adx_signal_func()
    volume_prof_signal_func()
    vwap_signal_func()
    ma12_21_signal_func()
    sma25_signal_func()
    ma7_16_signal_func()
    cci_kp_signal_func()
    rsi_delta_func()
    stoch1_adx_signal_func()
    cci_adx_signal_func()
    rsi_divergent_sto_signal_func()
    rsi_divergent_adx_signal_func()
    ADX_DMI_signal_func()
    DMI_signal_func()
    implement_adx_signal_func()
    renko_macdindian_signal_func()
    obv_signal_changes_func()
    obv_rsi_signal_changes_func()
    volume_prof_adx_signal_func()
    atr_obv_signal_func()
    atr_rsi_signal_func1()
    atr_rsi_signal_func2()
    atr_macd_crossover_signal_func()
    atr_ma25_signal_func()
    cci_ma5_signal_func()

    signalss={'adx_signal':adx_signal,'macd_crossover_signal':macd_crossover_signal,
            'ma30_200_signal':ma30_200_signal,'ma50_200_signal':ma50_200_signal,
            'ma30_50_signal':ma30_50_signal,'cdc_signal':cdc_signal,'sar_signal':sar_signal,
            'roc_signal':roc_signal,'obv_signal':obv_signal,'rsi_signal':rsi_signal,
            'bb_signal':bb_signal,'wpr_signal':wpr_signal,'stoch_signal':stoch_signal,
            'cci_signal':cci_signal,'atr_signal':atr_signal,'aroon_signal':aroon_signal,
            'psar_changetrend_signal':psar_changetrend_signal,'stoch_signal_2':stoch_signal_2,'rsiandmacd':rsiandmacd_signal,
            'renko_signal':renko_signal,'renko_macd_crossover_signal':renko_macd_crossover_signal,
            'fake_obv_renko_signal':fake_obv_renko_signal,'ma50_100_signal':ma50_100_signal,
            'fake_obv_macd_crossover_signal':fake_obv_macd_crossover_signal,'sma100_signal':sma100_signal,
            'ma30_100_signal':ma30_100_signal,'stoch2_adx_signal':stoch2_adx_signal,
            'bb_rsi_signal':bb_rsi_signal,'psar_wpr_signal':psar_wpr_signal,
            'cdc_atr_signal':cdc_atr_signal,'ma5_10_signal':ma5_10_signal,'macd_sto_signal':macd_sto_signal,
            'sto_ma50_200_signal':sto_ma50_200_signal,'atr_adx_signal':atr_adx_signal,
            'ma25_89_signal':ma25_89_signal,'ma21_89_signal':ma25_89_signal
            ,'macd_overzero_signal':macd_overzero_signal,'macd_kongpop_signal'
            :macd_kongpop_signal,'rsi_divergent_signal':rsi_divergent_signal,
            'adx_sto_kp_signal':adx_sto_kp_signal,'sma89_signal':sma89_signal,
            'real_obv_macd_crossover_signal':real_obv_macd_crossover_signal,'rsi_adx_signal':rsi_adx_signal,
            'renko_macdzero_signal':renko_macdzero_signal,'psar_adx_signal':psar_adx_signal,
            'cci_renko_signal':cci_renko_signal,'macd_crossover_wpr_signal':macd_crossover_wpr_signal,
            'aroon_kp_signal':aroon_kp_signal,'renko_adx_signal':renko_adx_signal,'psar_trend_signal':psar_trend_signal,
            'volume_prof_signal':volume_prof_signal,'vwap_signal':vwap_signal,'sma25_signal':sma25_signal,'ma12_21_signal':ma12_21_signal,
            'ma7_16_signal':ma7_16_signal,'cci_kp_signal':cci_kp_signal,'rsi_delta_signal':rsi_delta_signal,'stoch1_adx_signal':stoch1_adx_signal,
            'cci_adx_signal':cci_adx_signal,'rsi_divergent_sto_signal':rsi_divergent_sto_signal,
            'rsi_divergent_adx_signal':rsi_divergent_adx_signal,'ADX_DMI_signal':ADX_DMI_signal,'DMI_signal':DMI_signal,'implement_adx_signal':implement_adx_signal
            ,'renko_macdindian_signal':renko_macdindian_signal,'obv_signal_changes_signal':obv_signal_changes_signal,
            'obv_rsi_signal':obv_rsi_signal,'obv_ma30_signal':obv_ma30_signal,'volume_prof_adx_signal':volume_prof_adx_signal,'atr_obv_signal':atr_obv_signal,
            'atr_rsi_signal1':atr_rsi_signal1,'atr_rsi_signal2':atr_rsi_signal2,'atr_macd_crossover_signal':atr_macd_crossover_signal,'atr_ma25_signal':atr_ma25_signal,
            'cci_ma5_signal':cci_ma5_signal,'close_price':dfa1['Close']
           ,'adj_close_price':dfa1['Adj Close']}

    globals()['df_of_'+Real_name+'_signal'+'_train']=pd.DataFrame(signalss)


In [ ]:
# us_stock_list=['BA','FDX','AMP','COP','AAPL','TSLA','OXY','XOM','MCD','DIS']
# us_stock_list+=['NFLX','PXD','AMZN','META','TROW','TSN','AMD','BABA','CRSP','EA']
# us_stock_list+=['NKE','KO','INTC','F','CAT','SBUX','JNJ','QCOM','ADBE','ORCL']
'''dow30=['AXP','AMGN','AAPL','BA','CAT','CSCO','CVX',
'GS','HD','HON','IBM','INTC','JNJ'
,'KO','JPM','MCD','MMM','MRK','MSFT','NKE','PG','TRV',
'UNH','CRM','VZ','V','WMT','DIS','DOW']
us_stock_list=dow30
ticker_list=[]
for i,value in enumerate(us_stock_list):
    ticker_list.append(us_stock_list[i])
for i in range(len(us_stock_list)):
    findindi(ticker_list[i],us_stock_list[i],2019,1,2,2023,5,1) #15-19 18-22
    # globals()['df_of_'+us_stock_list[i]+'_signal_train'].to_csv(us_stock_list[i]+'dump1_1522_years.csv')
# thai_stock_list=['CPALL','MINT','HANA', 'BTS','BDMS', 'FORTH','JASIF', 'SCC','CPF','KBANK']'''
# thai_stock/_list+=['AOT','CHG','CPN','LPN','IRPC','IT','ADVANC','BBL','BANPU','COM7']
# thai_stock_list+=['GULF','JMART','TU','RCL','JAS','BGRIM','TRUE','SINGER','ORI','PTT']
set50=["AOT","BDMS","ORI","INTUCH","ADVANC","MEGA","SJWD","STA","CHG","DELTA"]
ticker_list=[]

set100=['AAV','ADVANC','AMATA','AP','BANPU','BBL','BCH','BCP','BCPG',
        'BDMS','BEC','BEM','BGRIM','BH','BLA','BTS','BYD','CBG','CENTEL','CHG','CK','CKP','CPALL',
        'CPN','DELTA','EA','EGCO','EPG','GLOBAL','GPSC','GULF',
'GUNKUL','HMPRO','INTUCH','IVL','JAS','JMART','JMT','KBANK','KCE','KKP',
'KTB','KTC','MEGA','MTC','NEX','ORI','PLANB','PSL','PTG',
'PTT','PTTEP','PTTGC','QH','RATCH','RCL','SAWAD','SCC','SINGER',
'SIRI','SJWD','SPALI','SPRC','TCAP','THANI','TISCO','TOP','TRUE'
,'TTB','TU','VGI','WHA','MINT','CPF','STA','COM7','AOT','LH']
UP = ['AOT', 'BDMS', 'ORI', 'INTUCH', 'ADVANC', 'MEGA', 'SJWD', 'STA', 'CHG', 'DELTA']
DOWN = ['MINT', 'TTB', 'BANPU', 'SIRI', 'KBANK', 'PTG', 'BLA', 'AAV', 'JAS', 'CK']
SIDEWAY = ['CPALL', 'BEM', 'AMATA', 'BCH', 'PTTGC', 'SPALI', 'TCAP', 'CENTEL', 'VGI', 'IVL']
#ขาด 'KEX','ONEE','OR','TIDLOR','TIPH','TQM'(ไม่มีข้อมูล)'ACE' 'AWC' 'BAM' 'CRC' 'DOHOME' 'RBF' 'SABUY''SCGP' 'STGT' 'OSP' 'THG'
## ปัญหา 'SCB'
thai_stock_list=set100
thai_stock_list=['AOT','BDMS','ORI','AAV', 'SIRI', 'BANPU','CENTEL', 'BCH', 'SPALI']
for i in range(len(thai_stock_list)):
    ticker_list.append(thai_stock_list[i]+'.BK')
for i,value in enumerate(thai_stock_list):
    print(set100[i])
    findindi(ticker_list[i],thai_stock_list[i],2022,9,22,2023,8,31)
    # globals()['df_of_'+thai_stock_list[i]+'_signal_train'].to_csv(thai_stock_list[i]+'dump1_1522_years.csv')

AAV
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-3-06c7f67dd3e9>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
<ipython-input-3-06c7f67dd3e9>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
<ipython-input-3-06c7f67dd3e9>:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'][ll_idx] = 1
<ipython-input-3-06c7f67dd3e9>:165: SettingWithCopyWarning: 
A value is trying to be set on a 

ADVANC
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-3-06c7f67dd3e9>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
<ipython-input-3-06c7f67dd3e9>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
<ipython-input-3-06c7f67dd3e9>:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'][ll_idx] = 1
<ipython-input-3-06c7f67dd3e9>:165: SettingWithCopyWarning: 
A value is trying to be set on a 

AMATA
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

<ipython-input-3-06c7f67dd3e9>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
<ipython-input-3-06c7f67dd3e9>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
<ipython-input-3-06c7f67dd3e9>:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'][ll_idx] = 1
<ipython-input-3-06c7f67dd3e9>:165: SettingWithCopyWarning: 
A value is trying to be set on a 

BCP
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-3-06c7f67dd3e9>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
<ipython-input-3-06c7f67dd3e9>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
<ipython-input-3-06c7f67dd3e9>:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'][ll_idx] = 1
<ipython-input-3-06c7f67dd3e9>:165: SettingWithCopyWarning: 
A value is trying to be set on a 

BCPG
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-3-06c7f67dd3e9>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][hh_idx] = 1
<ipython-input-3-06c7f67dd3e9>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_highs'][lh_idx] = -1
<ipython-input-3-06c7f67dd3e9>:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{key}_lows'][ll_idx] = 1
<ipython-input-3-06c7f67dd3e9>:165: SettingWithCopyWarning: 
A value is trying to be set on a 

In [ ]:
len(set100)

10

In [ ]:
##test without sliding
all_stock_list=thai_stock_list
for i,value in enumerate(all_stock_list):
    globals()[all_stock_list[i]+'_train'] = globals()['df_of_'+all_stock_list[i]+'_signal_train']
all_ticker_list=[]
for index,value in enumerate(all_stock_list):
    all_ticker_list.append(locals()[all_stock_list[index]+'_train'])
for i in range(len(thai_stock_list)):
    dfo=all_ticker_list[i]
    for j in range(77):
        globals()['sl'+str(j+1)+all_stock_list[i]]=dfo[dfo.columns.values[j]].round(0).astype(int)

In [ ]:
for j in range(len(thai_stock_list)): #เเต่ละหุ้น
    globals()['return_of_'+all_stock_list[j]+'_in_each_signal']=[]
def simulate_one_signal(all_ticker_list):
    global ret_l,ret_BH
    ret_l = [0] * 77
    ret_BH = 0
    for i in range(1,76): #เเต่ละ signal
        for j in range(len(thai_stock_list)): #เเต่ละหุ้น
            dfo=all_ticker_list[j]
            sl=(globals()['sl'+str(i)+all_stock_list[j]]).round(0).astype(int)
            ret=0
            sumall=0
            amount=0
            mon=100000
            if (i==1):
                stocknum = mon/dfo['close_price'][0]
                ret_BH += ((stocknum*0.998*dfo['close_price'][len(dfo)-1])+(mon-stocknum*dfo['close_price'][0]*1.002))
            for m in range(0,len(dfo)): #เเต่ละวัน
                r = sl[m]
                if r==1:
                    sumall=mon//dfo['close_price'][m]
                    amount+=sumall
                    mon-=sumall*dfo['close_price'][m]*1.002
                elif r==-1:
                    mon+=amount*dfo['close_price'][m]*0.998
                    amount=0
                else:
                    pass
            ret=mon+amount*dfo['close_price'][len(dfo)-1]
            globals()['return_of_'+all_stock_list[j]+'_in_each_signal'].append(((ret-100000)/1000))
            ret_l[i-1]+=ret

In [ ]:
simulate_one_signal(all_ticker_list)
def Convert(lst,j):
    res_dct = {j[i]:lst[i] for i in range(77)}
    return res_dct

sorted_footballers_by_goals = sorted(Convert(ret_l,dfo.columns).items(), key=lambda x:(x[1]))
converted_dict_ret = dict(sorted_footballers_by_goals)

print(converted_dict_ret)

{'close_price': 0, 'adj_close_price': 0, 'DMI_signal': 820302.3392596833, 'renko_macd_crossover_signal': 820910.2001019726, 'sma25_signal': 836406.8542461741, 'sma100_signal': 840457.2759349255, 'aroon_signal': 842196.0327425175, 'obv_signal_changes_signal': 843211.990793081, 'macd_crossover_signal': 848067.7595295589, 'atr_ma25_signal': 848075.103886673, 'psar_trend_signal': 848572.3013280217, 'atr_obv_signal': 848588.9160047157, 'psar_changetrend_signal': 852271.2304273809, 'psar_wpr_signal': 852271.2304273809, 'sar_signal': 853201.0388148496, 'sma89_signal': 853894.8374783512, 'roc_signal': 854127.1182408587, 'ma12_21_signal': 862613.1748596844, 'ma7_16_signal': 862613.1748596844, 'cci_kp_signal': 863933.5191078944, 'aroon_kp_signal': 865408.0986784822, 'vwap_signal': 869797.6520837105, 'real_obv_macd_crossover_signal': 871274.9077881925, 'rsi_divergent_signal': 875858.1272250537, 'ma30_50_signal': 877582.0283443998, 'cdc_signal': 878231.4722935578, 'renko_macdzero_signal': 880343.5

In [ ]:
len(all_stock_list)

78

In [ ]:
freq_l = [0] * 75
for i in range(1,76):
    freq=0
    for j in range(9): #เเต่ละหุ้น
    	a=0
    	labels = globals()['sl'+str(i)+all_stock_list[j]].value_counts().keys()
    	values = globals()['sl'+str(i)+all_stock_list[j]].value_counts().values
    	for index,l in enumerate(labels):
            if (l==0):
                a=values[index]
            else:
                a=a
    	freq+=sum(values)-a
    freq_l[i-1]=freq

In [ ]:
def Convert(lst,j):
    res_dct = {j[i]:lst[i] for i in range(75)}
    return res_dct

sorted_footballers_by_goals = sorted(Convert(freq_l,dfo.columns).items(), key=lambda x:x[1])
converted_dict_freq = dict(sorted_footballers_by_goals)

print(converted_dict_freq)

{'atr_signal': 0, 'cdc_atr_signal': 0, 'macd_sto_signal': 0, 'sto_ma50_200_signal': 0, 'atr_adx_signal': 0, 'macd_kongpop_signal': 0, 'rsi_delta_signal': 0, 'cci_ma5_signal': 0, 'cci_renko_signal': 2, 'rsi_divergent_sto_signal': 8, 'bb_rsi_signal': 13, 'atr_rsi_signal2': 13, 'adx_signal': 20, 'implement_adx_signal': 20, 'ma30_200_signal': 21, 'ma50_200_signal': 21, 'stoch2_adx_signal': 23, 'rsi_adx_signal': 24, 'ma30_100_signal': 35, 'obv_signal': 37, 'renko_macd_crossover_signal': 37, 'ma50_100_signal': 37, 'cci_signal': 38, 'cci_kp_signal': 38, 'bb_signal': 43, 'ma25_89_signal': 43, 'ma21_89_signal': 43, 'rsi_divergent_adx_signal': 46, 'wpr_signal': 53, 'cci_adx_signal': 53, 'renko_adx_signal': 54, 'ma30_50_signal': 57, 'stoch_signal': 61, 'rsiandmacd': 69, 'renko_macdindian_signal': 73, 'macd_crossover_wpr_signal': 87, 'atr_macd_crossover_signal': 91, 'ADX_DMI_signal': 98, 'aroon_signal': 104, 'fake_obv_renko_signal': 109, 'renko_macdzero_signal': 114, 'sma100_signal': 118, 'obv_rsi

In [ ]:
def sort_dict_by_value(d, reverse = False):
    return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))
p=sort_dict_by_value(store, True)

NameError: ignored

In [ ]:
dictionaries = [d1, d2, d3]  # List of dictionaries
summed_dict = {}  # Dictionary to store the summed values

for dictionary in dictionaries:
    for key, value in dictionary.items():
        if key in summed_dict:
            summed_dict[key] += value
        else:
            summed_dict[key] = value
sorted_dict = dict(sorted(summed_dict.items(), key=lambda x: x[1], reverse=True))
sorted_dict

{'rsiandmacd': 5443703.240034329,
 'macd_overzero_signal': 5193135.312393685,
 'atr_macd_crossover_signal': 4877970.654822198,
 'cdc_signal': 4874754.003598325,
 'cci_adx_signal': 4751738.250111887,
 'renko_macdzero_signal': 4732973.659753165,
 'atr_rsi_signal2': 4644512.681770934,
 'volume_prof_signal': 4329202.3186966805,
 'rsi_divergent_sto_signal': 4245025.489458555,
 'volume_prof_adx_signal': 4172833.7227278557,
 'ma50_100_signal': 4134285.4631373575,
 'ma50_200_signal': 4087064.996257836,
 'atr_ma25_signal': 4037671.9998066016,
 'roc_signal': 4032831.6563975476,
 'sma100_signal': 4013625.1826251755,
 'stoch2_adx_signal': 3993832.4538375656,
 'ADX_DMI_signal': 3981398.332998186,
 'ma30_200_signal': 3964645.516686335,
 'obv_signal_changes_signal': 3923011.9291733527,
 'ma30_100_signal': 3908153.545856004,
 'fake_obv_macd_crossover_signal': 3890460.96695374,
 'cci_renko_signal': 3873779.650719175,
 'stoch1_adx_signal': 3859505.14105946,
 'ma12_21_signal': 3805560.0510263457,
 'ma7_1

In [ ]:
'rsi_adx_signal': 10009180.521434778,'cci_adx_signal': 13835295.579421751

8431151.943425154

In [ ]:
{'vwap_signal': 682756.8997897124, 'stoch1_adx_signal': 771137.8370415523, 'renko_signal': 777875.2095228719, 'ma5_10_signal': 796392.2096054311, 'obv_ma30_signal': 817382.2105424643, 'psar_changetrend_signal': 818321.2079848424, 'psar_wpr_signal': 818321.2079848424, 'macd_crossover_signal': 825509.7527140127, 'psar_trend_signal': 840668.5800291663, 'aroon_signal': 842917.2658188266, 'stoch_signal_2': 853945.5359100284, 'obv_signal_changes_signal': 856669.3403450622, 'cci_kp_signal': 878107.2417638345, 'wpr_signal': 882583.2837545709, 'rsi_divergent_adx_signal': 882826.4720232239, 'bb_rsi_signal': 903007.8369880251, 'sma100_signal': 906587.5758823348, 'obv_signal': 907629.3806176701, 'ma30_50_signal': 926786.8520891975, 'adx_sto_kp_signal': 933976.87179174, 'sma89_signal': 936293.175138483, 'cci_adx_signal': 937053.8358260355, 'ma50_100_signal': 937563.017847023, 'stoch_signal': 940629.9418759748, 'roc_signal': 940931.9734600065, 'renko_macd_crossover_signal': 942007.0810302857, 'sto_ma50_200_signal': 950328.2748856478, 'ma30_100_signal': 955489.4061301433, 'sma25_signal': 961675.891743371, 'bb_signal': 974358.7592383203, 'ma30_200_signal': 986876.4742528296, 'macd_crossover_wpr_signal': 988853.6713826333, 'ma25_89_signal': 996456.8758976384, 'ma21_89_signal': 996456.8758976384, 'atr_signal': 1000000.0, 'cdc_atr_signal': 1000000.0, 'macd_sto_signal': 1000000.0, 'atr_adx_signal': 1000000.0, 'macd_kongpop_signal': 1000000.0, 'rsi_delta_signal': 1000000.0, 'cci_ma5_signal': 1000000.0, 'volume_prof_adx_signal': 1001174.6060852174, 'real_obv_macd_crossover_signal': 1002056.8744803113, 'renko_macdindian_signal': 1004897.6677769546, 'volume_prof_signal': 1018937.3490122538, 'renko_macdzero_signal': 1020659.4333109579, 'atr_rsi_signal1': 1021701.5329742751, 'psar_adx_signal': 1041150.4495942296, 'cci_signal': 1041556.7972421473, 'renko_adx_signal': 1042387.4618964033,
'sar_signal': 1049762.1788255896, 'ADX_DMI_signal': 1053964.503004096, 'ma12_21_signal': 1055676.317503478, 'ma7_16_signal': 1055676.317503478, 'rsi_signal': 1066896.0422388688, 'adx_signal': 1071269.230623189, 'implement_adx_signal': 1071269.230623189, 'fake_obv_macd_crossover_signal': 1095613.2101175203, 'atr_macd_crossover_signal': 1102124.975995627, 'rsi_divergent_signal': 1111331.7837568624, 'fake_obv_renko_signal': 1114246.9210761185, 'ma50_200_signal': 1115951.5680001555, 'atr_obv_signal': 1119848.6813311353, 'DMI_signal': 1124081.1353592812, 'aroon_kp_signal': 1135566.591422888, 'obv_rsi_signal': 1143352.6358696348, 'rsi_divergent_sto_signal': 1150975.06452911, 'macd_overzero_signal': 1152634.7165904474, 'stoch2_adx_signal': 1177622.0877392436, 'cdc_signal': 1207812.3915541351, 'rsi_adx_signal': 1243043.407069964, 'atr_ma25_signal': 1257198.8786557724, 'cci_renko_signal': 1295878.667101349, 'rsiandmacd': 1315253.5853614358, 'atr_rsi_signal2': 1609545.2583317976}

In [ ]:
(7303026.156030304-7800000)/78000

-6.371459538073029

In [ ]:
itt

Date
2016-01-04    9.10
2016-01-05    8.95
2016-01-06    9.00
2016-01-07    8.85
2016-01-08    8.85
              ... 
2023-04-24    9.80
2023-04-25    9.70
2023-04-26    9.75
2023-04-27    9.80
2023-04-28    9.80
Name: Close, Length: 1780, dtype: float64

In [ ]:
signalz=wpr_signal
buy_idx,sell_idx=[],[]
for idx,i in enumerate(signalz):
    if i==1:
        buy_idx.append(idx)
    if i==-1:
        sell_idx.append(idx)
il=[]
for i in range(len(signalz)):
    il.append(i)
# importing the module
import collections
frequency = collections.Counter(signalz)
print(dict(frequency))
from matplotlib.pyplot import figure
figure(figsize=(15, 6), dpi=720)
plt.plot(il,df_of_LH_signal_train['close_price'],marker='s',
    markersize=7,markerfacecolor='g',markevery=buy_idx)
plt.plot(il,df_of_LH_signal_train['close_price'],marker='v',
    markersize=7,markerfacecolor='r',markevery=sell_idx)
plt.show()

NameError: ignored

In [ ]:
freq_l = [0] * 52
for i in range(1,53):
    freq=0
    for j in range(60): #เเต่ละหุ้น
    	a=0
    	labels =globals()['sl'+str(i)+all_stock_list[j]].value_counts().keys()
    	values =globals()['sl'+str(i)+all_stock_list[j]].value_counts().values
    	for index,l in enumerate(labels):
            if (l==0):
                a=values[index]
            else:
                a=a
    	freq+=sum(values)-a
    freq_l[i-1]=freq

In [ ]:

def Convert(lst,j):
    res_dct = {j[i]:lst[i]/63579 for i in range(52)}
    return res_dct

sorted_footballers_by_goals = sorted(Convert(freq_l,dfo.columns).items(), key=lambda x:x[1])
converted_dict_freq = dict(sorted_footballers_by_goals)

print(converted_dict_freq)

NameError: ignored

In [ ]:
for i in converted_dict_ret:
    print(converted_dict_ret[str(i)])

7900000.0
7900000.0
7900000.0
7900000.0
7900000.0
7947118.043687119
7964866.74825983
7998941.59060073
8070704.104087768
8073288.502208983
8297181.376079717
8365276.237394483
8420117.986815825
8484987.451353833
8491904.599321168
8517731.305382727
8546741.404317103
8562336.056638794
8640026.177790724
8640026.177790724
8646560.64316393
8658388.403247446
8754631.072178753
8820295.520468086
8888569.31326766
8903492.002279565
8968635.65434452
9189954.216636041
9189954.216636041
9221627.339855962
9241513.959391633
9271547.41698572
9295918.662726238
9318138.7389344
9344754.848389512
9375498.742482781
9406199.95236284
9473382.622537483
9534757.9565889
9569237.446495231
9580459.30103139
9600081.472896326
9611044.089108767
9611044.089108767
9678548.540626597
9708039.727602933
9734072.319169516
9850706.403943567
9889026.420797313
9979286.94118712
10015051.149893975
10150624.21410744
10541292.960672555
10670438.42623752
10821770.336352924
10955893.996196259
10973436.01121534
11103529.952660786
1123

In [ ]:
inter_com = {k:(converted_dict_ret[k]*converted_dict_freq[k]) for k in converted_dict_ret.keys() if k in converted_dict_freq}

In [ ]:
inter_com = sorted(inter_com.items(), key=lambda x:x[1])

In [ ]:
inter_com

[('atr_signal', 0.0),
 ('cdc_atr_signal', 0.0),
 ('macd_sto_signal', 0.0),
 ('atr_adx_signal', 0.0),
 ('macd_kongpop_signal', 0.0),
 ('sto_ma50_200_signal', 1249.959584719344),
 ('cci_renko_signal', 14598.757066103515),
 ('ma50_200_signal', 70496.0979850945),
 ('psar_adx_signal', 77328.76305692681),
 ('ma30_200_signal', 80788.41171298339),
 ('adx_signal', 83246.0844573639),
 ('bb_rsi_signal', 109558.09692694819),
 ('ma50_100_signal', 109887.96890064928),
 ('ma30_100_signal', 114257.66165439229),
 ('cci_signal', 120648.7595902612),
 ('ma25_89_signal', 127740.67863796662),
 ('ma21_89_signal', 127740.67863796662),
 ('wpr_signal', 156209.64085699784),
 ('stoch2_adx_signal', 156801.8945602468),
 ('obv_signal', 207079.7705983658),
 ('ma30_50_signal', 213480.00662734103),
 ('stoch_signal', 221890.73080648357),
 ('bb_signal', 236090.99351252118),
 ('rsiandmacd', 280122.1574136507),
 ('vwap_signal', 300238.53763684595),
 ('macd_crossover_wpr_signal', 308614.5381610845),
 ('renko_adx_signal', 32